# Parameter fitting and sensitivity analysis

In [ ]:
# import the required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Before we can do any calculations we once more need to import the file with the measured data.

In [ ]:
# read the data from excel using pandas
df = pd.read_excel("data/water_balance_data.xlsx", index_col=0, parse_dates=True)
df.head()

## Parameter optimization of the mass balance model

Below we'll estimate the parameters for the water balance model that we developed, specifically the pan factor and the rainfall chloride concentration. The code that we used before to calculate the chloride mass balance is:

In [ ]:
def get_conc_cl(data, pan_factor, Cl_rain):
    """get the chloride concentration over time

    Parameters
    ----------
    data : DataFrame
        DataFrame containing the columns area, volume, rain and evaporation
    pan_factor : float
        pan evaporation factor
    Cl_rain : float
        chloride concentration of the rain

    Returns
    -------
    df : pandas DataFrame
        dataframe with the calculated concentration as a column
    """
    Cl_0 = 20  # g/m^3 = mg/l

    df = (
        data.copy()
    )  # Create a local copy, making sure the original DataFrame stays intact

    df["P"] = df["area"] * df["rain"] / 1000.0
    df["E"] = df["area"] * df["evaporation"] / pan_factor / 1000.0
    df["dV"] = -df["volume"].diff(periods=-1)
    df["I"] = df["P"] - df["E"] - df["dV"]

    P = df["P"].to_numpy()
    I = df["I"].to_numpy()
    V = df["volume"].to_numpy()

    M_Cl_g = np.empty(len(df))
    conc_Cl = np.empty(len(df))

    for i, (Vi, Pi, Ii) in enumerate(zip(V, P, I)):
        if i == 0:  # First day
            M_Cl_g[0] = Vi * Cl_0
            conc_Cl[0] = M_Cl_g[0] / Vi  # Gives Cl_0 of course!
        else:
            M_Cl_g[i] = M_Cl_g[i - 1] + dM_P - dM_I
            conc_Cl[i] = M_Cl_g[i] / Vi

        dM_P = Cl_rain * Pi
        dM_I = conc_Cl[i] * Ii

    df["conc_Cl"] = conc_Cl

    return df

Let's plot the calculated chloride concentrations along with the measured one, as well as the residuals.

In [ ]:
# read the data from excel using pandas
df = pd.read_excel(
    "data/water_balance_data.xlsx",
    index_col=0,
    parse_dates=True,
)

# run the model
dfnew = get_conc_cl(data=df, Cl_rain=5, pan_factor=1.2)

# plot the data
fig, axs = plt.subplots(2, 1, figsize=(8, 4), sharex=True)
axs[0].plot(dfnew["conc_Cl"])
axs[0].plot(dfnew["Cl_sample"], marker="o")
axs[1].plot(dfnew["Cl_sample"] - dfnew["conc_Cl"], marker=".")
axs[1].set_xticklabels(axs[1].get_xticklabels(), rotation=30, ha="right");

As noted previously the result isn't too bad, except that there appears to be a diverging trend in time. Let's see if we can improve the fit by optizing the `pan_factor` and the rainfall chloride concentration `Cl_rain`. There are a number of different ways to do it but here we will demonstrate the `lmfit` package. The [`lmfit` package](https://github.com/lmfit/lmfit-py) provides a range of non-linear least-squares minimization and curve-fitting functions for Python. It extends the methods of ` scipy.optimize` and provides some of the functionality that [PEST](https://pesthomepage.org) also provides.

The fitting procedure involves the following steps:
1. Create a function to calculate the chloride concentration given the two parameters: pan factor and chloride concentration (see above).
2. Create an objective function in Python that will calculate the residuals from the parameters. This function will call the function in step 1.
3. Run the `lmfit.minimize` function to find the optimal values

The code cell above takes care of step 1 and defined the code for the chloride mass balance as a function. The objective function below calls get_conc_cl and returns the difference between the measured and calculated chloride concentrations (step 2):

In [ ]:
def obj_cl_compare(pars):
    """objective function, compares calculated chloride concentration with
    measurements.

    Parameters
    ----------
    pars : lmfit Parameters
        parameters with optimization settings

    Returns
    -------
    numpy array with differences measured - modelled concentrations
    """

    global df

    parvals = pars.valuesdict()
    pan_factor = parvals["pan_factor"]
    Cl_rain = parvals["Cl_rain"]

    dfnew = get_conc_cl(df, pan_factor, Cl_rain)

    # Most rows have nan values for chloride so use dropna() to avoid returning nan's
    return (dfnew["Cl_sample"] - dfnew["conc_Cl"]).dropna().values

Now we create the variables necessary for the LmFit minimizer. Specifically, we need to create a `Parameter` object, to which we provide information about different parameters. For more information, check out the [LmFit Documentation](https://lmfit.github.io/lmfit-py/)

In [ ]:
import lmfit  # Remember, you should move this import at the top

parameters = lmfit.Parameters()
parameters.add(
    "pan_factor",  # Name of the parameter
    value=1.2,  # Initial parameter guesstimate
    min=0.1,  # Lower parameter bound
    max=3.0,  # Upper parameter bound
)
parameters.add(
    "Cl_rain",
    value=5.0,
    min=0.00001,
    max=20.0,
)

# Create a minimizer object
mini = lmfit.Minimizer(
    userfcn=obj_cl_compare,  # Function that returns the residuals
    params=parameters,  # lmfit parameter object
    calc_covar=True,  # Bool whether to compute the covariance matrix
)

# Run the minimize method (estimate the parameters)
result = mini.minimize(method="leastsq")
result

## Model sensitivity and uncertainty analysis

This remainder of this notebook demonstrates a few different ways in which the effect of model parameters on the model outcomes can be quantified using Python. It is not a formal discussion of model sensitivity and uncertainty analysis. The main aim is to demonstrate the flexibility of Python when it comes to running models mulitple times, and to show the effect on the model outcomes. Parameters that are considered to be uncertain are the pan factor, the rainfall chloride concentration and the measured rainfall, evaporation and water volume.

We will draw a number of samples from a normal distribution that respresent the uncertainty of the pan factor. The true value of the pan factor that was used to create the synthetic sample points was 1.3, which we'll use as the mean. Note that a slightly higher value was found during the calibration with `lmfit`, and that the standard error (i.e. the standard deviation) was reported to be approximately 0.064. In the example below we'll use a higher value, simply because it is then easier to see the differences between the model runs.

The code below uses the [`scipy.stats`](https://docs.scipy.org/doc/scipy/reference/stats.html) module to define a normal probability distribution with a mean $\mu_p$ and a standard deviation $\sigma_p$. The value of variable `N` defines the number of values that will be used to draw the probability density function. The `norm` function is used to create the normal distribution and is stored as `dist_p`. The values for the pan factors on the x axis are defined by first calculating the value that corresponds to the 1 and 99 percent percentiles. These values are used as limits for the range of `N` x values. The probability densities for `xp` are stored in `yp` and are obtained using the `pdf` method of `dist_p`.

In [ ]:
from scipy.stats import norm

N = 1000

mu_p = 1.3  # Mean pan evaporation factor
sigma_p = 0.1  # Standard error of the parameter

dist_p = norm(loc=mu_p, scale=sigma_p)

# Get the percentile values
x001 = dist_p.ppf(0.01)
x099 = dist_p.ppf(0.99)
print(f"The pan factor that corresponds to the 1% percentile is {x001}")
print(f"The pan factor that corresponds to the 99% percentile is {x099}")

# Define the values for the x axis
xp = np.linspace(
    x001,
    x099,
    N,
)

# Use the values in xp to get the corresponding probability densities
yp = dist_p.pdf(xp)

# Plot the probability density curve
plt.plot(xp, yp)
plt.xlabel("Pan factor")
plt.ylabel("Probability density");

The `cdf` method of `dist_p` returns the cummulative probabilities and can be used to draw the cummulative probability distribution.

In [ ]:
ypc = dist_p.cdf(xp)
plt.plot(xp, ypc)
plt.xlabel("Pan factor")
plt.ylabel("Probability");

Let's draw five pan factors from the distribution. The selected values roughly encompass the 95% and 68% confidence intervals and the median average is also included.

In [ ]:
sample_percentiles = [0.025, 0.16, 0.5, 0.84, 0.975]
pan_factors = dist_p.ppf(sample_percentiles)
print(pan_factors)
plt.plot(xp, ypc)
plt.plot(pan_factors, sample_percentiles, "C0o")
plt.xlabel("Pan factor")
plt.ylabel("Probability");

With the five different realizations of the pan factor, its effect on the model outcome can be visualized. All it takes is a short `for` loop that steps through the selected pan factors and calls the `get_conc` function each time, and then plots the calculated chloride concentrations as a function of time. The last line of code also draws the sample data points.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 2))

for pan_factor in pan_factors:
    dfnew = get_conc_cl(data=df, pan_factor=pan_factor, Cl_rain=4)
    ax.plot(dfnew["conc_Cl"])

plt.plot(df["Cl_sample"], marker="o")
plt.legend(["{:.1%}".format(p) for p in sample_percentiles])
plt.xticks(rotation=30, ha="right");

***Exercise 1***: Draw five values from a normal distribution of rainfall chloride concentrations (choose a mean and standard deviation that you think are reasonable). Plot the cummulative probability distribution.

In [ ]:
# Type your code here (hint: copy the code for the pan factor and adapt it for rainfall chloride)

Let's now run the model again, varying both the pan factor and the rainfall chloride concentration at the same time. The thing we need to remember though is that the realizations of both the pan factors and the rainfall chloride concentrations were drawn in the same order, that is the values in `pan_factors` and `Cl_rains` are both in increasing order. To fix this, we can use the `shuffle` function from `numpy.random`, which puts the array elements in random order. The result is illustrated below by printing the arrays to the screen before and after `shuffle` was called.

In [ ]:
print(pan_factors)
np.random.shuffle(pan_factors)
print(pan_factors)

print(Cl_rains)
np.random.shuffle(Cl_rains)
print(Cl_rains)

Let's rerun the model but now with two parameters (the pan factor and the rainfall chloride concentration) being uncertain and plot the result. Python's `zip` function comes in handy here because it allows us to loop over the contents of two lists simultaneously.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 2))
for pan_factor, Cl_rain in zip(pan_factors, Cl_rains):
    dfnew = get_conc_cl(data=df, pan_factor=pan_factor, Cl_rain=Cl_rain)
    ax.plot(dfnew["conc_Cl"])
ax.plot(df["Cl_sample"], marker="o");

***Exercise (homework)***: The first code cell below takes into consideration the uncertainty of the rainfall measurements. A ~10% random error (assumed to be normally distributed) is added to each value in the `rain` column of `df`. The 10% is approximate, it is the result of using a standard deviation of 0.033 for the normal distribution (with about 99.7% of values being within three standard deviations from the mean).

The second code cell creates a plot with one line for each model run. Because our model runs very fast we can easily run it 100 times. The lines plot close to one another, which resembles an uncertainty band (in the next session we'll look into how we can improve the appearance of graphs like this one).

Modify the code below to create a series of 100 model runs in which the pan factor, rainfall chloride concentration as well as the measured rainfall, evaporation and the dam water volume are all stochastic parameters. Your code will be a mix of the code examples above and the one provided below.

In [ ]:
def get_conc_cl(data, pan_factor=1.3, Cl_rain=2, sd1=0.033):
    Cl_0 = 20  # g/m^3 = mg/l

    df = data.copy()

    e_rain = np.random.normal(0, sd1, len(df["rain"])) * df["rain"]
    df["rain"] = df["rain"] + e_rain
    df["evaporation"] = df["evaporation"]
    df["volume"] = df["volume"]

    df["P"] = df["area"] * df["rain"] / 1000.0
    df["E"] = df["area"] * df["evaporation"] / (1000.0 * pan_factor)
    df["dV"] = -df["volume"].diff(periods=-1)
    df["I"] = df["P"] - df["E"] - df["dV"]

    M_Cl_g = np.empty(len(df))
    conc_Cl = np.empty(len(df))

    P = df["P"].to_numpy()
    I = df["I"].to_numpy()
    V = df["volume"].to_numpy()
    for i, (Vi, Pi, Ii) in enumerate(zip(V, P, I)):
        if i == 0:  # First day
            M_Cl_g[0] = Vi * Cl_0
            conc_Cl[0] = M_Cl_g[0] / Vi  # Gives Cl_0 of course!
        else:
            M_Cl_g[i] = M_Cl_g[i - 1] + dM_P - dM_I
            conc_Cl[i] = M_Cl_g[i] / Vi

        dM_P = Cl_rain * Pi
        dM_I = conc_Cl[i] * Ii

    return df

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
for n in range(100):
    df = get_conc_cl(df.copy(), sd1=0.033)
    ax.plot(dfnew["conc_Cl"])
ax.plot(df["Cl_sample"], marker="o");